In [ ]:
#IMPORT PACAKGES
import pandas as pd
import random
import time
import numpy as np
import httplib2
import yaml
import json
import sys
import subprocess
from googleapiclient import discovery
from oauth2client import client

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, ElementNotInteractableException, TimeoutException
from selenium.webdriver.chrome.options import Options

In [ ]:
#DEFINE FUNCTIONS
print('Define functions...')

def get_config(config_set):
    cred_path = 'id_gsheet.yml'
    config = yaml.safe_load(open(cred_path))
    config_set = config['id'][config_set]
    return config_set



def get_credentials(cred_set, json_configs=''):
    credentials = None
    if json_configs != '':
        credentials = get_credentials_from_json(json_configs)
    else:
        client_secret = cred_set['client_secret']
        grant_type = cred_set['grant_type']
        refresh_token = cred_set['refresh_token']
        client_id = cred_set['client_id']
        url = cred_set['url']
        data = client_secret + '&' + grant_type + '&' + refresh_token + '&' + client_id
        response = json.loads(subprocess.check_output(['curl', '--data', data, url]))
        access_token = response["access_token"]
        credentials = client.AccessTokenCredentials(access_token, 'my-user-agent/1.0')
        http = httplib2.Http()
        http = credentials.authorize(http)
    return credentials



def click_elem(tag_name, elem_name, try_count):
    switcher = {
        "ID": By.ID,
        "NAME": By.NAME,
        "CLASS_NAME": By.CLASS_NAME,
        "XPATH": By.XPATH,
        "CSS_SELECTOR": By.CSS_SELECTOR,
        "TAG_NAME": By.TAG_NAME
    }
    try:
        element = WebDriverWait(driver, 10).\
            until(EC.element_to_be_clickable((switcher.get(tag_name), elem_name)))
        try:
            element.click()
        except StaleElementReferenceException:
            if try_count > 9:
                print("\nPage reloaded more than 10 times!")
                print("\nProgram terminated!")
                sys.exit()
            print("Error: Element '" + elem_name + "' not attached to the page document!")
            return "Timeout"
        except ElementNotInteractableException:
            if try_count > 9:
                print("\nPage reloaded more than 10 times!")
                print("\nProgram terminated!")
                sys.exit()
            print("Error: Element '" + elem_name + "' not interactable!")
            return "Timeout"
    except TimeoutException:
        if try_count > 9:
            print("\nPage reloaded more than 10 times!")
            print("\nProgram terminated!")
            sys.exit()
        print("Error: Element '" + elem_name + "' not clickable!")
        return "Timeout"



def injection(spreadsheetId, df, rangeName):
    #generate dataframe
    df.fillna('')
    df = df.astype(str)
    df.replace(['NaN'],'',inplace= True)
    df_columns = df.columns.values.tolist()
    df_content = df.values.tolist()
    df_content.insert(0, df_columns)

    cred_set =  get_config('spreadsheet')
    credentials = get_credentials(cred_set)
    service = discovery.build('sheets', 'v4', credentials=credentials)
    list = df_content
    resource = {
      "majorDimension": "ROWS",
      "values": list
    }
    #delete all the input
    service.spreadsheets().values().clear(
        spreadsheetId=spreadsheetId,
        range=rangeName
    ).execute()
    #add all input
    service.spreadsheets().values().append(
      spreadsheetId=spreadsheetId,
      range=rangeName,
      body=resource,
      valueInputOption="USER_ENTERED"
    ).execute()



def read_gsheet(file_id_properti, rangeName):
    cred_set =  get_config('spreadsheet')
    credentials = get_credentials(cred_set)
    service = discovery.build('sheets', 'v4', credentials=credentials)
    content = service.spreadsheets().values().get(spreadsheetId=file_id_properti, range=rangeName).execute()
    #data transform from gsheet to dataframe
    headers = list(content['values'])[0]
    df = pd.DataFrame(data=np.zeros((0,len(headers))), columns=headers)
    result_df = pd.DataFrame(content['values'],columns = headers)
    df  = pd.concat([df ,result_df],axis=0)
    df = df.iloc[1:].reset_index(drop = True)
    return df



def login(try_count):
    print('Go to '+website+'...')
    driver.get(website)
    current_page_url = driver.current_url
    print('Landed on '+current_page_url+'!')
    time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC 

    if 'public/login' in current_page_url:
        while True:
            print('Initiating logging in to the website...')
            if wait_for_visibility('XPATH', "//input[@placeholder='Email address']", try_count) != 'Timeout':

                driver.find_element(By.XPATH, "//input[@placeholder='Email address']").send_keys(id_innoscripta)
                print(driver.find_element(By.XPATH, "//input[@placeholder='Email address']").get_attribute('value'))
                time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC

                driver.find_element(By.XPATH, "//input[@placeholder='Password']").send_keys(pass_innoscripta)
                print(driver.find_element(By.XPATH, "//input[@placeholder='Password']").get_attribute('value'))
                time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC

                print(driver.find_element(By.XPATH, "//button[@type='submit']").get_attribute('type'))
                click_elem('XPATH', '//button[@type="submit"]', try_count)
                try_count = 0
                time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC

                print('Login successful!')
                current_page_url = driver.current_url
                print('Landed on '+current_page_url+'!')
                break

            else:
                print('Element not found!')
                print('Refreshing page...')
                try_count += 1
                driver.get(current_page_url)
                time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC
    else:
        print('Logged in status detected, go to https://platform.globaldatabase.com/app-aggregator/prospect/companies')
        driver.get('https://platform.globaldatabase.com/app-aggregator/prospect/companies')
        current_page_url = driver.current_url
        time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC

    return try_count
    


def filter_canada(try_count):
    while True:
        print('Filter country for Canada...')
        if wait_for_visibility('XPATH', "//span[contains(text(),'Location')]", try_count) != 'Timeout':
            print('Clicking on Location filter...')
            click_elem('XPATH', "//span[contains(text(),'Location')]", try_count)
            print('Element: '+driver.find_element(By.XPATH, "//span[contains(text(),'Location')]").text+' has been clicked!')
            try_count = 0
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC
            break
        else:
            print('Refreshing page...')
            try_count += 1
            try_count = login(try_count)

    while True:
        print('Clicking on Countries filter...')
        if wait_for_visibility('XPATH', "//div[contains(text(),'Countries')]", try_count) != 'Timeout':
            click_elem('XPATH', "//div[contains(text(),'Countries')]", try_count)
            print('Element: '+driver.find_element(By.XPATH, "//div[contains(text(),'Countries')]").text+' has been clicked!')
            try_count = 0
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC
            break
        else:
            print('Try clicking on Location filter again...')
            try_count += 1
            click_elem('XPATH', "//span[contains(text(),'Location')]", try_count)
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC

    while True:
        print('Searching for Canada...')
        if wait_for_visibility('XPATH', "//input[@placeholder='Search country']", try_count) != 'Timeout':
            driver.find_element(By.XPATH, "//input[@placeholder='Search country']").clear()
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC
            driver.find_element(By.XPATH, "//input[@placeholder='Search country']").send_keys('Canada')
            print('Text inserted: '+driver.find_element(By.XPATH, "//input[@placeholder='Search country']").get_attribute('value'))
            try_count = 0
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC
            break
        else:
            print('Try clicking on Countries filter again...')
            try_count += 1
            click_elem('XPATH', "//div[contains(text(),'Countries')]", try_count)
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC

    while True:
        print('clicking on Canada...')
        if wait_for_visibility('XPATH', "//span[contains(text(), 'Canada')]", try_count) != 'Timeout':
            click_elem('XPATH', "//span[@class='ant-tree-checkbox']", try_count)
            try_count = 0
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC
            active_filter = []
            for element in driver.find_elements(By.CLASS_NAME, 'text-content'):
                active_filter.append(element.text)
            if 'Canada' in active_filter:
                print('Country Canada has been selected!')
                break
        else:
            print('Try saerching for Canada again...')
            try_count += 1
            driver.find_element(By.XPATH, "//input[@placeholder='Search country']").clear()
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC
            driver.find_element(By.XPATH, "//input[@placeholder='Search country']").send_keys('Canada')
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC

    while True:
        print('Remove status filter: active...')
        if wait_for_visibility('XPATH', "//span[contains(text(),'Companies')]", try_count) != 'Timeout':
            print('Clicking on Companies filter...')
            click_elem('XPATH', "//span[contains(text(),'Companies')]", try_count)
            print('Element: '+driver.find_element(By.XPATH, "//span[contains(text(),'Companies')]").text+' has been clicked!')
            try_count = 0
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC
            break
        else:
            print('Refreshing page...')
            try_count += 1
            driver.get(current_page_url)
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC
            if 'public/login' in driver.current_url:
                try_count = login(try_count)

    while True:
        print('Clicking on Status filter...')
        if wait_for_visibility('XPATH', "//div[contains(text(),'Status')]", try_count) != 'Timeout':
            click_elem('XPATH', "//div[contains(text(),'Status')]", try_count)
            print('Element: '+driver.find_element(By.XPATH, "//div[contains(text(),'Status')]").text+' has been clicked!')
            try_count = 0
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC
            break
        else:
            print('Try clicking on Companies filter again...')
            try_count += 1
            click_elem('XPATH', "//span[contains(text(),'Companies')]", try_count)
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC

    while True:
        print('Deselect Active status...')
        if wait_for_visibility('XPATH', "//span[contains(text(), 'Active')]", try_count) != 'Timeout':
            click_elem('XPATH', "//span[contains(text(), 'Active')]", try_count)
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC
            active_filter = []
            for element in driver.find_elements(By.CLASS_NAME, 'text-content'):
                active_filter.append(element.text)
            if 'Active' not in active_filter:
                print('Status Active has been removed!')
                try_count = 0
                break
        else:
            print('Try clicking on Status filter again...')
            try_count += 1
            click_elem('XPATH', "//div[contains(text(),'Status')]", try_count)
            time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC

    return try_count

In [ ]:
#SETUP SELENIUM PYTHON
print('Setting up Selenium...')
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 1})
driver = webdriver.Chrome(options=chrome_options)
print('Selenium is ready to be used!')

In [ ]:
#LOGIN TO GLOBAL DATABASE AND SET FILTER CANADA
id_innoscripta = {put your user id here}
pass_innoscripta = {put your password here}
website = 'https://platform.globaldatabase.com/public/login'
current_page_url = website
random_sleep = [2,2.5,3,3.5,4,4.5,5]
try_count = 0

try_count = login(try_count)
try_count = filter_canada(try_count)

In [ ]:
#NUMBER OF COMPANIES LISTED AS CANADIAN COMPANIES
num_companies = int(driver.find_element(By.CLASS_NAME, 'counts-highlighted-text').text.replace(',',''))

#CLICKING ON "view-more" BUTTON TO BE DONE MANUALLY
for i in np.arange(0, num_companies):
    click_elem('CLASS_NAME', 'view-more', try_count)
    time.sleep(3)
    
len(driver.find_elements(By.CLASS_NAME, 'column-logotype'))

In [ ]:
#GET COMPANY LIST ON THE MAIN PAGE TABLE
companies = []
while len(companies) == 0:
    elem = driver.find_elements(By.XPATH, "//a[@class='column-logotype']")
    for i in np.arange(0, len(elem)):
        companies.append([elem[i].text.split('\n')[1], elem[i].get_attribute('href')])
    time.sleep(random.choice(random_sleep)) # RANDOMLY DELAYING ACTIVITY TO MAKE IT LESS ROBOTIC

print('Companies URL successfully obtained!')
print('Number of companies to be scrapped: '+str(len(companies)))

file_id = '1cgSJetgMDgn0nmo0cf2bgfn84KfS4rU6udMlLheEb-4'

#STORE COMPANY LIST TO GSHEET
print('Store company list to gsheet...')
companies_df_gsheet = read_gsheet(file_id, 'Company List!A1:AZ')
companies_name = companies_df_gsheet.company_name.to_list()
companies_gsheet = companies_df_gsheet.values.tolist()
# only store data which are not already in the gsheet
for item in companies:
    if item[0] not in companies_name:
        companies_gsheet.append(item)
df = pd.DataFrame(companies_gsheet, columns=['company_name', 'company_url'])
injection(file_id, df, 'Company List!A1:AZ')